In [39]:
import functools
import sys
import scipy.io
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp

import Test_Likelihood as tl
import ODE_Dynamics as od
import Positive_Symptom_fn as fn

from functools import partial

tfd = tfp.distributions
tfb = tfp.bijectors

In [40]:
data = scipy.io.loadmat('../data/test_data_simulation.mat')
x = tf.cast(data['data_simulation'],dtype = tf.float32)
test_data = np.reshape(
      x, x.shape)
test_data = tf.transpose(test_data)
#print(test_data)
#test_data = test_data[0:15,0:3]

In [41]:
test_data

<tf.Tensor: shape=(149, 3), dtype=float32, numpy=
array([[0.000e+00, 1.060e+03, 3.000e+00],
       [1.000e+00, 1.062e+03, 8.000e+00],
       [2.000e+00, 1.037e+03, 3.000e+00],
       [3.000e+00, 9.970e+02, 6.000e+00],
       [4.000e+00, 9.850e+02, 6.000e+00],
       [5.000e+00, 9.920e+02, 5.000e+00],
       [6.000e+00, 9.670e+02, 4.000e+00],
       [7.000e+00, 1.068e+03, 2.000e+00],
       [8.000e+00, 1.060e+03, 6.000e+00],
       [9.000e+00, 1.011e+03, 5.000e+00],
       [1.000e+01, 1.049e+03, 7.000e+00],
       [1.100e+01, 1.001e+03, 8.000e+00],
       [1.200e+01, 9.740e+02, 6.000e+00],
       [1.300e+01, 9.500e+02, 9.000e+00],
       [1.400e+01, 9.470e+02, 8.000e+00],
       [1.500e+01, 9.740e+02, 8.000e+00],
       [1.600e+01, 1.003e+03, 1.000e+01],
       [1.700e+01, 1.028e+03, 9.000e+00],
       [1.800e+01, 1.014e+03, 1.100e+01],
       [1.900e+01, 1.026e+03, 1.200e+01],
       [2.000e+01, 9.730e+02, 1.300e+01],
       [2.100e+01, 1.036e+03, 1.000e+01],
       [2.200e+01, 1.016e+

In [42]:
def joint_log_prob(test_data, R0, nu):
    vdyn_ode_fn = od.ViralDynamics
    positive_fn = fn.proba_pos_sym(170306.4 * 1E-5).positive_fn
    symptom_fn = fn.proba_pos_sym(170306.4 * 1E-5).symptom_fn
    prob_s_ibar = 0.1
    mu_b, sigma_b = 5, 1
    beta = np.random.normal(mu_b, sigma_b, 1)
    L = 0.0025/beta
    V0 = 1E3
    X0 = 1E6
    Y0 = V0
    par=tf.constant(np.array([[L,0.01,beta*1E-7,0.5,20,10, V0, X0, Y0]], dtype=np.float32))
    vpar = par
    pospar = par
    sympar = par
    sample_size = 1166
    k = 1
    index = 1
    mu_b, sigma_b = 5, 1
    beta = np.random.normal(mu_b, sigma_b, 1)   #"rate at which virus infects host cells"
    L = 0.0025/beta
    par=np.array([[L,0.01,beta*1E-7,0.5,20.0,10.0]])
    V0 = np.random.normal(1E3, 1E2, 1)
    X0 = 1E6
    Y0 = V0
    init_state=(np.array([[V0,X0,Y0]], dtype=np.float32))

    while index <= sample_size - 1:
        beta = np.random.normal(mu_b, sigma_b, 1)   #"rate at which virus infects host cells"
        L = 0.0025/beta
        par_new=np.array([[L,0.01,beta*1E-7,0.5,20.0,10.0]])
        par = np.concatenate((par, par_new), axis = 0)
        V0 = np.random.normal(1E3, 1E2, 1)
        X0 = 1E6
        Y0 = V0
        init_state_new=(np.array([[V0,X0,Y0]], dtype=np.float32))
        init_state = np.concatenate((init_state, init_state_new), 0)

        index +=1
        

    vpar = tf.constant(par, dtype=tf.float32)
    pospar = par
    sympar = par

    epipar = tf.constant(np.array([[R0, 5.0E-08,nu, 0.001, 0.999]], dtype=np.float32))

    loglike = tl.loglik(test_data, vdyn_ode_fn, positive_fn, symptom_fn, prob_s_ibar, prob_fp=0.0, Epi_Model=od.SIR,
                 duration= 20.0, Epi_cadence=0.5, Vir_cadence=0.0625)
    ll,pp = loglike.__call__(test_data, epipar, vpar, pospar, sympar)
    return ll

In [65]:
ll = joint_log_prob(test_data, R0 = 1.8, nu = 0.1)
print(ll)

unnormalized_posterior_log_prob = partial(joint_log_prob, test_data)

initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
tf.Tensor(-43749.324, shape=(), dtype=float32)


In [66]:
# Initialize parameters
r0 = 1.5; nu0 = 0.14
initial_state = [r0, nu0]
unconstraining_bijectors = [
    tfb.Log(),
    tfb.Invert(tfb.Sigmoid())
    ]

In [69]:

#@tf.function(autograph=False)
def sample():
    return tfp.mcmc.sample_chain(
        num_results = 10,
        num_burnin_steps = 5,
        current_state = initial_state,
        kernel = tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(
                inner_kernel = tfp.mcmc.HamiltonianMonteCarlo(
                    target_log_prob_fn = unnormalized_posterior_log_prob,
                     step_size = 0.1,
                     num_leapfrog_steps = 2),
                bijector = unconstraining_bijectors),
             num_adaptation_steps = 3),
        trace_fn=lambda _, pkr: pkr.inner_results.inner_results.is_accepted)

In [ ]:
[R0, nu], is_accepted = sample()


initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
final time
tf.Tensor(149.0, shape=(), dtype=float32)
initial time
0.0
fin

In [16]:

# Compute posterior means of Epidemic parameters

acceptance_rate = tf.reduce_mean(tf.cast(is_accepted, dtype=tf.float32)).numpy()
mean_R0 = tf.reduce_mean(R0, axis=0).numpy()
mean_nu = tf.reduce_mean(nu, axis=0).numpy()
mean_i = tf.reduce_mean(i, axis=0).numpy()
mean_s = tf.reduce_mean(s, axis=0).numpy()

NameError: name 'is_accepted' is not defined

In [17]:
print('acceptance_rate:', acceptance_rate)
print('avg basic reproduction number:', mean_R0)
print('avg recovery rate:', mean_nu)
print('avg infected', mean_i)
print('avg susceptible', mean_s)

NameError: name 'acceptance_rate' is not defined